In [9]:
import pandas as pd
import numpy as np

train=pd.read_excel('Project - 4 - Train Data.xlsx')
test=pd.read_excel('Project - 4 - Test Data.xlsx')
y_train=train[['CPL_Status']]
x_train=train.drop(['Loanapp_ID','Sex','Marital_Status','first_name','last_name','email','address','INT_ID','Prev_ID','AGT_ID','CPL_Status'], axis=1)
x_test=test.drop(['Loanapp_ID','Sex','Marital_Status','first_name','last_name','email','address','INT_ID','Prev_ID','AGT_ID'], axis=1)

print("No of Missing Values : \n",x_train.isna().sum())
x_train

No of Missing Values : 
 Dependents      15
Qual_var         0
SE              32
App_Income_1     0
App_Income_2     0
CPL_Amount       2
CPL_Term        14
Credit_His      50
Prop_Area        0
dtype: int64


,Dependents,Qual_var,SE,App_Income_1,App_Income_2,CPL_Amount,CPL_Term,Credit_His,Prop_Area
0,0,Grad,N,7018.8,0.0,NaN,360.0,1.0,Urban
1,1,Grad,N,5499.6,1809.6,153.6,360.0,1.0,Rural
2,0,Grad,Y,3600.0,0.0,79.2,360.0,1.0,Urban
3,0,Non Grad,N,3099.6,2829.6,144.0,360.0,1.0,Urban
4,0,Grad,N,7200.0,0.0,169.2,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...
609,0,Grad,N,3480.0,0.0,71.0,360.0,1.0,Rural
610,3+,Grad,N,4927.2,0.0,40.0,180.0,1.0,Rural
611,1,Grad,N,9686.4,288.0,253.0,360.0,1.0,Urban
612,2,Grad,N,9099.6,0.0,187.0,360.0,1.0,Urban


# Missing Values treatment

In [10]:
#unprovided data is better to filled with -1 than to it fill with 0 because 0 means there is no dependencies
#which is logically diffrent than unprovided dependencies.
x_train['Dependents']=x_train['Dependents'].fillna(-1)

#for easy interpretation assume all 3+ dependents to 4
x_train['Dependents'][x_train['Dependents']=='3+']=4

x_train['SE']=x_train['SE'].fillna('N')
x_train['CPL_Amount']=x_train['CPL_Amount'].fillna(0)

x_train['CPL_Term'].fillna((x_train['CPL_Term'].median()), inplace=True)
x_train['Credit_His']=x_train['Credit_His'].fillna(-1)
x_train['tot_income']=x_train['App_Income_1']+x_train['App_Income_2']

#Same missing values treatment for Test data
x_test['Dependents']=x_test['Dependents'].fillna(-1)
[x_test['Dependents']=='3+'] is True
x_test['Dependents'][x_test['Dependents']=='3+']= 4
x_test['SE']=x_test['SE'].fillna('N')
x_test['CPL_Amount']=x_test['CPL_Amount'].fillna(0)
x_test['CPL_Term'].fillna((x_test['CPL_Term'].median()), inplace=True)
x_test['Credit_His']=x_test['Credit_His'].fillna(-1)
x_test['tot_income']=x_test['App_Income_1']+x_train['App_Income_2']

x_test.isna().sum()
print("Missing Values in y_train : ",y_train.isna().sum())

x_train

Missing Values in y_train :  CPL_Status    0
dtype: int64


C:\Users\Bhupati\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Bhupati\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Dependents,Qual_var,SE,App_Income_1,App_Income_2,CPL_Amount,CPL_Term,Credit_His,Prop_Area,tot_income
0,0,Grad,N,7018.8,0.0,0.0,360.0,1.0,Urban,7018.8
1,1,Grad,N,5499.6,1809.6,153.6,360.0,1.0,Rural,7309.2
2,0,Grad,Y,3600.0,0.0,79.2,360.0,1.0,Urban,3600.0
3,0,Non Grad,N,3099.6,2829.6,144.0,360.0,1.0,Urban,5929.2
4,0,Grad,N,7200.0,0.0,169.2,360.0,1.0,Urban,7200.0
...,...,...,...,...,...,...,...,...,...,...
609,0,Grad,N,3480.0,0.0,71.0,360.0,1.0,Rural,3480.0
610,4,Grad,N,4927.2,0.0,40.0,180.0,1.0,Rural,4927.2
611,1,Grad,N,9686.4,288.0,253.0,360.0,1.0,Urban,9974.4
612,2,Grad,N,9099.6,0.0,187.0,360.0,1.0,Urban,9099.6


# Categorical Data Encoding

In [11]:
#using LabelEncoder if we encode 'Qual_var' then 'Grad' are assigned 0 and 'Non Grad' are assinged 1
#But logically 'Grad' have better chance of getting CPL than 'Non Grad'
x_train['Qual_var'][x_train['Qual_var']=='Grad']=1
x_train['Qual_var'][x_train['Qual_var']=='Non Grad']=0
x_test['Qual_var'][x_test['Qual_var']=='Grad']=1
x_test['Qual_var'][x_test['Qual_var']=='Non Grad']=0

from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
x_train['SE'] =encoder.fit_transform(x_train['SE'])
x_train['Prop_Area'] =encoder.fit_transform(x_train['Prop_Area'])
y_train['CPL_Status'] =encoder.fit_transform(y_train['CPL_Status'])

x_test['SE'] =encoder.fit_transform(x_test['SE'])
x_test['Prop_Area'] =encoder.fit_transform(x_test['Prop_Area'])

x_train['CPL_Status']= y_train
x_train

C:\Users\Bhupati\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Bhupati\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Bhupati\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

,Dependents,Qual_var,SE,App_Income_1,App_Income_2,CPL_Amount,CPL_Term,Credit_His,Prop_Area,tot_income,CPL_Status
0,0,1,0,7018.8,0.0,0.0,360.0,1.0,2,7018.8,1
1,1,1,0,5499.6,1809.6,153.6,360.0,1.0,0,7309.2,0
2,0,1,1,3600.0,0.0,79.2,360.0,1.0,2,3600.0,1
3,0,0,0,3099.6,2829.6,144.0,360.0,1.0,2,5929.2,1
4,0,1,0,7200.0,0.0,169.2,360.0,1.0,2,7200.0,1
...,...,...,...,...,...,...,...,...,...,...,...
609,0,1,0,3480.0,0.0,71.0,360.0,1.0,0,3480.0,1
610,4,1,0,4927.2,0.0,40.0,180.0,1.0,0,4927.2,1
611,1,1,0,9686.4,288.0,253.0,360.0,1.0,2,9974.4,1
612,2,1,0,9099.6,0.0,187.0,360.0,1.0,2,9099.6,1


# Feature Scaling for continuous data 

In [12]:
from sklearn.preprocessing import MinMaxScaler

def scale_columns(x,columns):
    for i in columns:
        x[i] =pd.DataFrame(MinMaxScaler().fit_transform(pd.DataFrame(x[i])),columns=[i])
        
    return x

x_train=scale_columns(x_train,['App_Income_1','App_Income_2','CPL_Amount','CPL_Term','tot_income'])
x_test=scale_columns(x_test,['App_Income_1','App_Income_2','CPL_Amount','CPL_Term','tot_income'])
x_train['Qual_var']=pd.to_numeric(x_train['Qual_var'],errors='coerce')
cor=x_train.corr()
print("Correlation between columns of x_train and y_train :\n")
min(cor.iloc[:-1,-1])
cor.iloc[:-1,-1]


Correlation between columns of x_train and y_train :



Qual_var        0.085884
SE             -0.003700
App_Income_1   -0.004710
App_Income_2   -0.059187
CPL_Amount      0.034665
CPL_Term       -0.022549
Credit_His      0.280181
Prop_Area       0.032112
tot_income     -0.031271
Name: CPL_Status, dtype: float64

In [13]:
x_train

,Dependents,Qual_var,SE,App_Income_1,App_Income_2,CPL_Amount,CPL_Term,Credit_His,Prop_Area,tot_income,CPL_Status
0,0,1,0,0.070489,0.000000,0.000000,0.743590,1.0,2,0.055394,1
1,1,1,0,0.054830,0.036192,0.182857,0.743590,1.0,0,0.058435,0
2,0,1,1,0.035250,0.000000,0.094286,0.743590,1.0,2,0.019583,1
3,0,0,0,0.030093,0.056592,0.171429,0.743590,1.0,2,0.043980,1
4,0,1,0,0.072356,0.000000,0.201429,0.743590,1.0,2,0.057292,1
...,...,...,...,...,...,...,...,...,...,...,...
609,0,1,0,0.034014,0.000000,0.084524,0.743590,1.0,0,0.018326,1
610,4,1,0,0.048930,0.000000,0.047619,0.358974,1.0,0,0.033485,1
611,1,1,0,0.097984,0.005760,0.301190,0.743590,1.0,2,0.086352,1
612,2,1,0,0.091936,0.000000,0.222619,0.743590,1.0,2,0.077189,1


# Support Vector Classifier

In [14]:
from sklearn.decomposition import PCA
#using Principal Component Analysis for dimentionality reduction to improve data quality.

pca= PCA(n_components=None)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
#X_test = pca.transform(x_test)
e=(pca.explained_variance_ratio_)
for i in e:
    print(i)
    
pca= PCA(n_components=4)
X_train = pca.fit_transform(x_train.iloc[:,:-1])

from sklearn.model_selection import train_test_split
#X = x_train[['Dependents', 'Qual_var','SE','App_Income_1','App_Income_2','CPL_Amount','CPL_Term','Credit_His','Prop_Area','tot_income']]
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)


0.5356329285125688
0.2165650221254018
0.13155936472804822
0.05875641572374176
0.04027363550843503
0.006482961756181036
0.005077978553876655
0.0038035706470966385
0.0018481224446500145
2.150305049729847e-34


In [15]:
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score

clf =svm.SVC(kernel='rbf',gamma='auto').fit(X_train,y_train)
y_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
clf.score(X_test, y_test)

[[14 19]
 [ 2 88]]


0.8292682926829268

In [16]:
pca= PCA(n_components=6)
X_train = pca.fit_transform(x_train.iloc[:,:-1])

from sklearn.model_selection import train_test_split
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

clf =svm.SVC(kernel='rbf',gamma='auto').fit(X_train,y_train)
y_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
clf.score(X_test, y_test)

[[16 17]
 [ 7 83]]


0.8048780487804879

In [17]:
pca= PCA(n_components=3)
X_train = pca.fit_transform(x_train.iloc[:,:-1])

#X = x_train[['Dependents', 'Qual_var','SE','App_Income_1','App_Income_2','CPL_Amount','CPL_Term','Credit_His','Prop_Area','tot_income']]
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)
clf =svm.SVC(kernel='rbf',gamma='auto').fit(X_train,y_train)
y_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
clf.score(X_test, y_test)

[[14 19]
 [ 2 88]]


0.8292682926829268

Therefore, the highest accuracy using SVC model is achhieved with 3 or 4 or 5 principal components,
i.e. 0.8292682926829268

# Logistic Regression

In [18]:
pca= PCA(n_components=3)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
clf.score(X_test, y_test)

[[ 4 29]
 [ 8 82]]


0.6991869918699187

In [19]:
pca= PCA(n_components=4)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
clf.score(X_test, y_test)

[[ 7 26]
 [ 7 83]]


0.7317073170731707

In [20]:
pca= PCA(n_components=6)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
clf.score(X_test, y_test)

[[ 6 27]
 [ 7 83]]


0.7235772357723578

# kNN

In [27]:
pca= PCA(n_components=6)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score

knn =KNeighborsClassifier(n_neighbors =11).fit(X_train, y_train)
y_pred =knn.predict(X_test)
print(confusion_matrix(y_test,y_pred))
knn.score(X_test, y_test)

[[14 19]
 [ 5 85]]


0.8048780487804879

In [28]:
pca= PCA(n_components=6)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

knn =KNeighborsClassifier(n_neighbors =15).fit(X_train, y_train)
y_pred =knn.predict(X_test)
print(confusion_matrix(y_test,y_pred))
knn.score(X_test, y_test)

[[14 19]
 [ 4 86]]


0.8130081300813008

Therefore, the best score using kNN model is achhieved with 6 principal components and 15 nearest neighbors
i.e.  0.8130081300813008

# Random Forest Classifier

In [29]:
pca= PCA(n_components=6)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

from sklearn.ensemble import RandomForestClassifier
forest= RandomForestClassifier(min_samples_split=25, random_state=0)

mod= forest.fit(X_train, y_train)
print(confusion_matrix(y_test,y_pred))
mod.score(X_test, y_test)


[[14 19]
 [ 4 86]]


0.8048780487804879

In [30]:
pca= PCA(n_components=7)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

forest= RandomForestClassifier(min_samples_split=25, random_state=0)

mod= forest.fit(X_train, y_train)
print(confusion_matrix(y_test,y_pred))
mod.score(X_test, y_test)


[[14 19]
 [ 4 86]]


0.8130081300813008

Therefore, the best score using Random Forest Classifier model is achhieved with 7 or 9 principal components
i.e.  0.8130081300813008

# XGB

In [31]:
pca= PCA(n_components=5)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=0)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
xgb_model.score(X_test, y_test)

[[17 16]
 [ 8 82]]


0.8048780487804879

In [32]:
pca= PCA(n_components=8)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=0)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
xgb_model.score(X_test, y_test)

[[18 15]
 [11 79]]


0.7886178861788617

Therefore, the best score using XGB model is achhieved with 3 or 4 or 5 principal components
i.e.  0.8048780487804879

# Decision Tree

In [34]:
pca= PCA(n_components=5)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.7642276422764228

In [35]:
pca= PCA(n_components=3)
X_train = pca.fit_transform(x_train.iloc[:,:-1])
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.8048780487804879

# Model Comparision

Therefore, the best score using Decision Tree model is achhieved with 3 principal components
i.e.  0.8048780487804879

# Real Time Predicion

In [47]:
##invoke the best ML classification

from sklearn.decomposition import PCA

pca= PCA(n_components=5)
X_train = pca.fit_transform(x_train.iloc[:,:-1])

from sklearn.model_selection import train_test_split
y = x_train['CPL_Status']
X_train, X_test, y_train, y_test = train_test_split(X_train, y, random_state=0,test_size=0.20)

from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score

clf =svm.SVC(kernel='rbf',gamma='auto').fit(X_train,y_train)
y_pred= clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
clf.score(X_test, y_test)

[[14 19]
 [ 2 88]]


0.8292682926829268

In [48]:
X = pca.transform(x_test)    # transform the data as x_train was transformed.
y_pred = clf.predict(X)      # predict the result

y_pred.reshape(367,1)
x_test

,Dependents,Qual_var,SE,App_Income_1,App_Income_2,CPL_Amount,CPL_Term,Credit_His,Prop_Area,tot_income
0,0,1,0,0.078865,0.000000,0.157088,0.746835,1.0,2,0.078865
1,1,1,0,0.042411,0.062500,0.187739,0.746835,1.0,2,0.063202
2,2,1,0,0.068938,0.075000,0.344828,0.746835,1.0,2,0.068938
3,2,1,0,0.032263,0.106083,0.137931,0.746835,-1.0,2,0.064774
4,0,0,0,0.045168,0.000000,0.095785,0.746835,1.0,2,0.045168
...,...,...,...,...,...,...,...,...,...,...
362,4,0,1,0.055274,0.074042,0.126756,0.746835,1.0,2,0.087441
363,0,1,0,0.057329,0.029542,0.129949,0.746835,1.0,2,0.099147
364,0,1,0,0.044810,0.083042,0.147510,0.746835,-1.0,1,0.044810
365,0,1,0,0.068938,0.099708,0.198595,0.746835,1.0,0,0.068938


In [49]:
test=pd.read_excel('Project - 4 - Test Data.xlsx')
test=test.drop(['Loanapp_ID','Sex','Marital_Status','first_name','last_name','email','address','INT_ID','Prev_ID','AGT_ID'], axis=1)
test['CPL_Status']= pd.DataFrame(y_pred)
print('Predicted result for Test Data :')
test

Predicted result for Test Data :


,Dependents,Qual_var,SE,App_Income_1,App_Income_2,CPL_Amount,CPL_Term,Credit_His,Prop_Area,CPL_Status
0,0,Grad,N,6864.0,0.0,132.0,360.0,1.0,Urban,1
1,1,Grad,N,3691.2,1800.0,151.2,360.0,1.0,Urban,1
2,2,Grad,N,6000.0,2160.0,249.6,360.0,1.0,Urban,1
3,2,Grad,N,2808.0,3055.2,120.0,360.0,NaN,Urban,1
4,0,Non Grad,N,3931.2,0.0,93.6,360.0,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...
362,3+,Non Grad,Y,4810.8,2132.4,113.0,360.0,1.0,Urban,1
363,0,Grad,N,4989.6,850.8,115.0,360.0,1.0,Urban,1
364,0,Grad,N,3900.0,2391.6,126.0,360.0,NaN,Semi U,1
365,0,Grad,N,6000.0,2871.6,158.0,360.0,1.0,Rural,1
